# OPTIMIZE Command

Delta Lake can improve the speed of read queries from a table by coalescing small files into larger ones

## Compaction (bin-packing)

Bin-packing optimization is **_idempotent_**, meaning that if it is run twice on the same dataset, the second run has no effect.

Bin-packing aims to produce **evenly-balanced data files** with respect to their size on disk, but not necessarily number of tuples per file. However, the two measures are most often correlated.


In [ ]:
%%sql
DROP TABLE IF EXISTS demo.optimize_demo;
CREATE TABLE demo.optimize_demo (id int);

In [ ]:
%%sql
INSERT INTO demo.optimize_demo VALUES(1);
INSERT INTO demo.optimize_demo VALUES(2);
INSERT INTO demo.optimize_demo VALUES(3);
INSERT INTO demo.optimize_demo VALUES(4);
INSERT INTO demo.optimize_demo VALUES(5);
INSERT INTO demo.optimize_demo VALUES(6);
INSERT INTO demo.optimize_demo VALUES(7);
INSERT INTO demo.optimize_demo VALUES(8);
INSERT INTO demo.optimize_demo VALUES(9);

> Looking at storage level

In [ ]:
mssparkutils.fs.ls("Tables/optimize_demo")

> Counting how many files there are

In [ ]:
sc.binaryFiles("Tables/optimize_demo/").count()

## Run the command

Delta Lake can **improve the speed of read** queries from a table by coalescing small files into larger ones.

In [ ]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forName(spark, 'demo.optimize_demo')
delta_table.optimize().executeCompaction() 

> OR

In [ ]:
%%sql
OPTIMIZE optimize_demo;

## Delta Log

OPTIMIZE operation is add to the delta log and you can track it

In [ ]:
display(delta_table.history())

In [ ]:
deltalog = spark.read.json("Tables/optimize_demo/_delta_log/00000000000000000010.json")
# Remove information
display(deltalog.select("remove.path").where("remove is not null"))

In [ ]:
# Add information
display(deltalog.select("add.path").where("add is not null"))

In [ ]:
display(spark.read.table("demo.optimize_demo"))

In [ ]:
%%sql
SELECT * FROM demo.optimize_demo

## Vacuum

In [ ]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")
delta_table.vacuum(0) 

In [ ]:
mssparkutils.fs.ls("Tables/optimize_demo")

In [ ]:
sc.binaryFiles("Tables/optimize_demo/").count()

In [ ]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "true")

## Subset of data

You must have a table partitioned to be able to run OPTIMIZE command in a subset of data

In [ ]:
from pyspark.sql.functions import expr, lit, col
from pyspark.sql.types import *


spark.sql("DROP TABLE IF EXISTS optimize_demo")

df = spark.range(0,10000) \
    .withColumn("date", expr("cast(concat('2023-06-', cast(rand(5) * 30 as int) + 1) as date)")) 

df.write.partitionBy("date").format("delta").saveAsTable("optimize_demo")

In [ ]:
delta_table.optimize().where("date='2023-06-01'").executeCompaction() 

In [ ]:
%%sql
OPTIMIZE optimize_demo WHERE date='2023-06-01'

# Clean up

In [ ]:
%%sql
DROP TABLE IF EXISTS demo.optimize_demo;